## Instrucciones
    - Carga de datos: Descarga el archivo de datos house_prices.csv que contiene las siguientes columnas:
        - SqFt: Tamaño de la casa en pies cuadrados.
        - Bedrooms: Número de habitaciones.
        - Bathrooms: Número de baños.
        - Price: Precio de venta de la casa.
    - Exploración de datos: Realiza un análisis exploratorio de los datos para entender su estructura y detectar posibles problemas como valores nulos o outliers.
    - Preparación de datos: Divide los datos en un conjunto de entrenamiento (80%) y un conjunto de prueba (20%).
    - Entrenamiento del modelo: Entrena un modelo de regresión lineal utilizando los datos de entrenamiento.
    - Evaluación del modelo: Evalúa el modelo utilizando el conjunto de prueba. Calcula el error cuadrático medio (MSE) y el R-cuadrado (R²) del modelo.
    - Predicción: Usa el modelo entrenado para predecir el precio de una casa con las siguientes características:
        - Tamaño: 2000 pies cuadrados
        - Habitaciones: 3
        - Baños: 2

In [1]:
# Paso 0: Importar las librerias a utilizar
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [13]:
# Paso 1: Cargar el dataset
df = pd.read_csv('../data/house-prices.csv')
print(df.shape)

# Paso 2: Exploración de datos
print("Primeras filas del dataset:")
print(df.head())
print("############################")
print("\nDescripción estadística:")
print(df.describe())
print("############################")

# Verificación de casos nulos
print("\nValores nulos por columna:")
print(df.isnull().sum())
print("############################")

# Manejo de valores nulos (si existen)
# df = df.dropna()  # Aquí los eliminamos; alternativamente podrías rellenar con medias o medianas

# Detección de outliers (opcional)
# Aquí puedes usar métodos como boxplots para detectar visualmente valores extremos, pero los omitiremos por simplicidad

(128, 8)
Primeras filas del dataset:
   Home   Price  SqFt  Bedrooms  Bathrooms  Offers Brick Neighborhood
0     1  114300  1790         2          2       2    No         East
1     2  114200  2030         4          2       3    No         East
2     3  114800  1740         3          2       1    No         East
3     4   94700  1980         3          2       3    No         East
4     5  119800  2130         3          3       3    No         East
############################

Descripción estadística:
             Home          Price         SqFt    Bedrooms   Bathrooms  \
count  128.000000     128.000000   128.000000  128.000000  128.000000   
mean    64.500000  130427.343750  2000.937500    3.023438    2.445312   
std     37.094474   26868.770371   211.572431    0.725951    0.514492   
min      1.000000   69100.000000  1450.000000    2.000000    2.000000   
25%     32.750000  111325.000000  1880.000000    3.000000    2.000000   
50%     64.500000  125950.000000  2000.000000    3

In [15]:
# Paso 3: Preparación de datos: Separar las variables predictoras (X) y la variable objetivo (y)
X = df[['SqFt', 'Bedrooms', 'Bathrooms']]
y = df['Price']

# Dividimos los datos en conjuntos de entrenamiento y prueba (80%-20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [16]:
# Paso 4: Entrenamiento del modelo
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [17]:
# Paso 5: Evaluación del modelo en el conjunto de prueba
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"\nError Cuadrático Medio (MSE): {mse}")
print(f"R-cuadrado (R²): {r2}")


Error Cuadrático Medio (MSE): 320149938.2302681
R-cuadrado (R²): 0.4568195777548879


In [19]:
# Paso 6: Predicción del precio de una casa con las características especificadas
nueva_casa = pd.DataFrame([[2000, 3, 2]], columns=['SqFt', 'Bedrooms', 'Bathrooms'])
prediccion_precio = model.predict(nueva_casa)

print(f"\nEl precio estimado para una casa de 2000 SqFt, 3 habitaciones y 2 baños es: ${prediccion_precio[0]:.2f}")



El precio estimado para una casa de 2000 SqFt, 3 habitaciones y 2 baños es: $123537.00
